In [13]:
import pandas as pd

# Load the CSV file
file_path = '/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/merged_csv_file_using_geopandas.csv'
df = pd.read_csv(file_path)

data = df
# Grouping the data by 'OBJECTID' and counting the crashes
crash_counts = data.groupby('OBJECTID').size()
# Creating a new dataframe with OBJECTID and the crash counts
pavement_section_data = data.drop_duplicates(subset='OBJECTID').set_index('OBJECTID')
pavement_section_data['Crash_Count'] = crash_counts

# Resetting the index to include OBJECTID as a column
pavement_section_data.reset_index(inplace=True)

# The final dataframe is stored in pavement_section_data

pavement_section_data["Crash_Rate"]= pavement_section_data["Crash_Count"] *100000000/(pavement_section_data["AADT"]*pavement_section_data["PMIS_LENGTH"]*365)

print(f"Total crashes is {pavement_section_data['Crash_Count'].sum()} ")



Total crashes is 21536 


/var/folders/yl/h3jt_tns2s14c8y5w9qptb6c0000gn/T/ipykernel_31733/1261433241.py:5: DtypeWarning: Columns (143,172,176,177,178,179,188,203,204,207,208,209,210,211,212,243,244,252,257,258,267,268,274,275,281) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [29]:
import pandas as pd
import statsmodels.formula.api as smf 
import statsmodels.api as sm # Using smf for concise formula-based modeling

# Load data (assuming it's in a CSV file)

# Fill missing values with 0
pavement_section_data.fillna(0, inplace=True)

# Create crash per segment feature
pavement_section_data["Crash_per_segment"] = pavement_section_data["Crash_Count"] / pavement_section_data["PMIS_LENGTH"]

model = smf.glm("Crash_per_segment ~ IRI + AADT + SPEED + PCI_2 + CRACK_INDX + FAULT_INDX + FRICT + FAULTAV + RUT + TRUCKS + WIDTH + PMIS_LENGTH",
    data=pavement_section_data,
    family=sm.families.NegativeBinomial(),).fit()

# Print model summary
print(model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:      Crash_per_segment   No. Observations:                 3308
Model:                            GLM   Df Residuals:                     3295
Model Family:        NegativeBinomial   Df Model:                           12
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -7382.3
Date:                Sun, 07 Jan 2024   Deviance:                       1378.7
Time:                        19:34:46   Pearson chi2:                 1.80e+03
No. Iterations:                    18   Pseudo R-squ. (CS):             0.6222
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.4655      0.291      8.485      

/Users/lebakuprathyushkumarreddy/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
